In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path = "/content/drive/MyDrive/Narasio/5. Products and ratings of fashion site/"
os.listdir(path)

['product_fashion_ratings.db',
 'Data Description Products and Ratings of Fashion Site.txt']

In [4]:
import sqlite3
import pandas as pd
import numpy as np
from numpy import random
from numpy import mean
from numpy.random import seed
import scipy.stats as stats
from scipy.stats import ttest_1samp, ttest_ind

# Library untuk visualisasi
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
conn = sqlite3.connect(path + "product_fashion_ratings.db")
cur = conn.cursor()

In [6]:
cur.execute('''SELECT
                NAME FROM
                sqlite_master
                WHERE
                type='table'
                ''')
cur.fetchall()

[('products',),
 ('brand_details',),
 ('main_categories',),
 ('sub_categories',),
 ('item_details',),
 ('events',)]

Memunculkan head (5 data teratas) dari masing-masing tabel, dan memanggil data framenya

In [7]:
df_products = pd.read_sql_query('''
                                  SELECT * FROM products
                                  ''', conn)
df_products.head(2)

,index,product_ID,product_name
0,0,1,Chapeau à Visière Anti-UV pour Protection Solaire
1,1,2,Écharpe longue 180cm à pois en broderie pour f...


In [8]:
df_brand_details = pd.read_sql_query('''
                                        SELECT * FROM
                                        brand_details
                                      ''', conn)
df_brand_details.head(2)

,index,brand_ID,brand_name,brand_url
0,0,1,Socofy-1,None
1,1,2,MEANBEAUTY,https://fr.newchic.com/meanbeauty-brand-1436.html


In [9]:
df_main_categories = pd.read_sql_query('''
                                          SELECT * FROM
                                          main_categories
                                        ''', conn)
df_main_categories.head(2)

,index,category_ID,category_name
0,0,1,accessories
1,1,2,bags


In [10]:
df_sub_categories = pd.read_sql_query('''
                                        SELECT * FROM
                                        sub_categories
                                      ''', conn)
df_sub_categories.head(2)                                  

,index,subcategory_ID,subcategory_name
0,0,1,Écriture & Peinture
1,1,2,Épilateur


In [11]:
df_item_details = pd.read_sql_query('''
                                      SELECT * FROM
                                      item_details
                                    ''', conn)
df_item_details.head(2)

,index,item_ID,product_ID,variation_0_color,variation_1_color
0,0,1,1,Bleu ciel,Gris
1,1,2,2,armée verte,Bleu


In [12]:
df_events = pd.read_sql_query('''
                                SELECT * FROM
                                events
                              ''', conn)
df_events.head(2)

,index,ID,sku,image_url,url_area,product_ID,brand_ID,category_ID,subcategory_ID,item_ID,current_price,perc.discount,likes_count,is_new,codCountry
0,0,27788,SKU585926,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/loskii-wall-art-4619/p-...,4636,120,4,21,4677,21.48 USD,42.0,285,0,"ID,MY,PH,SG,TH,VN"
1,1,42560,SKU028546,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/plus-size-blouses-and-s...,9862,396,9,98,10372,22.99 USD,65.0,147,0,"ID,MY,PH,SG,TH,VN"


# **Data Manipulation**


In [13]:
# tabel main categories, tidak ada missing value
df_main_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          9 non-null      int64 
 1   category_ID    9 non-null      int64 
 2   category_name  9 non-null      object
dtypes: int64(2), object(1)
memory usage: 344.0+ bytes


In [14]:
# mengubah datatype category_ID menjadi string
df_main_categories['category_ID'] = df_main_categories['category_ID'].astype(str)
df_main_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          9 non-null      int64 
 1   category_ID    9 non-null      object
 2   category_name  9 non-null      object
dtypes: int64(1), object(2)
memory usage: 344.0+ bytes


In [15]:
# pengecekan apakah terdapat duplicated values
df_main_categories[df_main_categories["category_ID"].duplicated(keep=False)]

,index,category_ID,category_name


In [16]:
# tabel sub categories, tidak ada missing value
df_sub_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   index             451 non-null    int64 
 1   subcategory_ID    451 non-null    int64 
 2   subcategory_name  451 non-null    object
dtypes: int64(2), object(1)
memory usage: 10.7+ KB


In [17]:
# mengubah datatype subcategory_ID menjadi string
df_sub_categories['subcategory_ID'] = df_sub_categories['subcategory_ID'].astype(str)
df_sub_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   index             451 non-null    int64 
 1   subcategory_ID    451 non-null    object
 2   subcategory_name  451 non-null    object
dtypes: int64(1), object(2)
memory usage: 10.7+ KB


In [18]:
# pengecekan apakah terdapat duplicated values
df_sub_categories[df_sub_categories["subcategory_ID"].duplicated(keep=False)]

,index,subcategory_ID,subcategory_name


In [19]:
# tabel products, tidak ada missing value
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13313 entries, 0 to 13312
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         13313 non-null  int64 
 1   product_ID    13313 non-null  int64 
 2   product_name  13313 non-null  object
dtypes: int64(2), object(1)
memory usage: 312.1+ KB


In [20]:
# mengubah datatype product_ID menjadi string
df_products['product_ID'] = df_products['product_ID'].astype(str)
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13313 entries, 0 to 13312
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         13313 non-null  int64 
 1   product_ID    13313 non-null  object
 2   product_name  13313 non-null  object
dtypes: int64(1), object(2)
memory usage: 312.1+ KB


In [21]:
# pengecekan apakah terdapat duplicated values
df_products[df_products["product_ID"].duplicated(keep=False)]

,index,product_ID,product_name


In [22]:
# tabel brand details, terdapat missing value
df_brand_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       419 non-null    int64 
 1   brand_ID    419 non-null    int64 
 2   brand_name  419 non-null    object
 3   brand_url   398 non-null    object
dtypes: int64(2), object(2)
memory usage: 13.2+ KB


In [23]:
# terdapat 21 missing values pada kolom brand_url
pd.isnull(df_brand_details).sum()

index          0
brand_ID       0
brand_name     0
brand_url     21
dtype: int64

In [24]:
# karena brand_url memiliki 21/419 missing values dan dirasa tidak diperlukan, maka kolom akan di drop
df_brand_details = df_brand_details.drop(['brand_url'], axis=1)
pd.isnull(df_brand_details).sum()

index         0
brand_ID      0
brand_name    0
dtype: int64

In [25]:
# mengubah datatype brand_ID menjadi string
df_brand_details['brand_ID'] = df_brand_details['brand_ID'].astype(str)
df_brand_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       419 non-null    int64 
 1   brand_ID    419 non-null    object
 2   brand_name  419 non-null    object
dtypes: int64(1), object(2)
memory usage: 9.9+ KB


In [26]:
# pengecekan apakah terdapat duplicated values
df_brand_details[df_brand_details.duplicated(keep=False)]

,index,brand_ID,brand_name


In [27]:
# tabel item details, terdapat missing value
df_item_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13989 entries, 0 to 13988
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   index              13989 non-null  int64 
 1   item_ID            13989 non-null  int64 
 2   product_ID         13989 non-null  int64 
 3   variation_0_color  11488 non-null  object
 4   variation_1_color  9385 non-null   object
dtypes: int64(3), object(2)
memory usage: 546.6+ KB


In [28]:
# terdapat 2501/13989(17%) missing values pada kolom variation_0_color dan 4604/13989(32%) pada kolom variation_1_color
hilang_var1 = pd.isnull(df_item_details).sum()
rasio_var1 = hilang_var1.div(len(df_item_details))
print(hilang_var1, rasio_var1)

index                   0
item_ID                 0
product_ID              0
variation_0_color    2501
variation_1_color    4604
dtype: int64 index                0.000000
item_ID              0.000000
product_ID           0.000000
variation_0_color    0.178783
variation_1_color    0.329116
dtype: float64


In [29]:
# karena jumlah missing values yang dibawah 60% maka akan drop baris yang missing
df_item_details = df_item_details.dropna(axis=0, how="any")
pd.isnull(df_item_details).sum()

index                0
item_ID              0
product_ID           0
variation_0_color    0
variation_1_color    0
dtype: int64

In [30]:
# mengubah datatype item_ID dan product_ID menjadi string
df_item_details['item_ID'] = df_item_details['item_ID'].astype(str)
df_item_details['product_ID'] = df_item_details['product_ID'].astype(str)
df_item_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9385 entries, 0 to 13987
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   index              9385 non-null   int64 
 1   item_ID            9385 non-null   object
 2   product_ID         9385 non-null   object
 3   variation_0_color  9385 non-null   object
 4   variation_1_color  9385 non-null   object
dtypes: int64(1), object(4)
memory usage: 439.9+ KB


In [31]:
# pengecekan apakah terdapat duplicated values
df_item_details[df_item_details["item_ID"].duplicated(keep=False)]

,index,item_ID,product_ID,variation_0_color,variation_1_color


In [32]:
# tabel events, terdapat missing value
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14161 entries, 0 to 14160
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           14161 non-null  int64  
 1   ID              14161 non-null  int64  
 2   sku             14161 non-null  object 
 3   image_url       14161 non-null  object 
 4   url_area        14161 non-null  object 
 5   product_ID      14161 non-null  int64  
 6   brand_ID        14161 non-null  int64  
 7   category_ID     14161 non-null  int64  
 8   subcategory_ID  14161 non-null  int64  
 9   item_ID         14161 non-null  int64  
 10  current_price   14151 non-null  object 
 11  perc.discount   14156 non-null  float64
 12  likes_count     14161 non-null  int64  
 13  is_new          14161 non-null  int64  
 14  codCountry      13018 non-null  object 
dtypes: float64(1), int64(9), object(5)
memory usage: 1.6+ MB


In [33]:
# terdapat 10 missing values pada kolom current_price, 5 values pada kolom perc.discount dan 1143 values pada kolom codCountry
hilang_events = pd.isnull(df_events).sum()
rasio_events = hilang_events.div(len(df_events))
print(hilang_events, rasio_events)

index                0
ID                   0
sku                  0
image_url            0
url_area             0
product_ID           0
brand_ID             0
category_ID          0
subcategory_ID       0
item_ID              0
current_price       10
perc.discount        5
likes_count          0
is_new               0
codCountry        1143
dtype: int64 index             0.000000
ID                0.000000
sku               0.000000
image_url         0.000000
url_area          0.000000
product_ID        0.000000
brand_ID          0.000000
category_ID       0.000000
subcategory_ID    0.000000
item_ID           0.000000
current_price     0.000706
perc.discount     0.000353
likes_count       0.000000
is_new            0.000000
codCountry        0.080715
dtype: float64


In [34]:
# Menghapus missing value dengan drop baris yang missing
df_events = df_events.dropna(axis=0, how="any")
pd.isnull(df_events).sum()

index             0
ID                0
sku               0
image_url         0
url_area          0
product_ID        0
brand_ID          0
category_ID       0
subcategory_ID    0
item_ID           0
current_price     0
perc.discount     0
likes_count       0
is_new            0
codCountry        0
dtype: int64

In [35]:
# mengubah datatype ID, product_ID, brand_ID, category_ID, subcategory_ID, item_ID, dan is_new menjadi string
df_events['ID'] = df_events['ID'].astype(str)
df_events['product_ID'] = df_events['product_ID'].astype(str)
df_events['brand_ID'] = df_events['brand_ID'].astype(str)
df_events['category_ID'] = df_events['category_ID'].astype(str)
df_events['subcategory_ID'] = df_events['subcategory_ID'].astype(str)
df_events['item_ID'] = df_events['item_ID'].astype(str)
df_events['is_new'] = df_events['is_new'].astype(str)
df_events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13004 entries, 0 to 14160
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           13004 non-null  int64  
 1   ID              13004 non-null  object 
 2   sku             13004 non-null  object 
 3   image_url       13004 non-null  object 
 4   url_area        13004 non-null  object 
 5   product_ID      13004 non-null  object 
 6   brand_ID        13004 non-null  object 
 7   category_ID     13004 non-null  object 
 8   subcategory_ID  13004 non-null  object 
 9   item_ID         13004 non-null  object 
 10  current_price   13004 non-null  object 
 11  perc.discount   13004 non-null  float64
 12  likes_count     13004 non-null  int64  
 13  is_new          13004 non-null  object 
 14  codCountry      13004 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 1.6+ MB


In [36]:
# pengecekan apakah terdapat duplicated values, ternyata ada 124 rows yang duplicate
df_events[df_events["ID"].duplicated(keep=False)]

,index,ID,sku,image_url,url_area,product_ID,brand_ID,category_ID,subcategory_ID,item_ID,current_price,perc.discount,likes_count,is_new,codCountry
11,11,78267,SKU039926,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/kingso-wall-lamps-5395/...,2326,119,4,225,2358,3.59 USD,47.0,2,0,"ID,MY,PH,SG,TH,VN"
12,12,78267,SKU039926,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/kingso-wall-lamps-5395/...,2326,119,4,225,2358,3.59 USD,47.0,2,0,"ID,MY,PH,SG,TH,VN"
275,275,1054376,SKU396836,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/topacc-handmade-toys-50...,2223,160,4,213,2255,25.75 USD,44.0,77,0,"ID,MY,PH,SG,TH,VN"
276,276,1054376,SKU396836,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/topacc-handmade-toys-50...,2223,160,4,213,2255,25.75 USD,44.0,77,0,"ID,MY,PH,SG,TH,VN"
655,655,1113128,SKU544412,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/socofy-flats-3615/p-111...,7590,20,8,142,7780,34.99 USD,49.0,12227,0,"ID,MY,PH,SG,TH,VN"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13141,13141,1692610,SKUE46513,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/xiaomi-smart-watches-49...,5136,12,5,251,5177,42.99 USD,57.0,27,0,"ID,MY,PH,SG,TH,VN"
13165,13165,1693840,SKUE46510,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/xiaomi-smart-watches-49...,5140,12,5,251,5181,49.99 USD,52.0,19,0,"ID,MY,PH,SG,TH,VN"
13166,13166,1693840,SKUE46510,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/xiaomi-smart-watches-49...,5140,12,5,251,5181,49.99 USD,52.0,19,0,"ID,MY,PH,SG,TH,VN"
13313,13313,1703260,SKUF48706,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/lostisy-womens-sandals-...,7584,378,8,353,7772,23.6 USD,51.0,144,0,"ID,MY,PH,SG,TH,VN"


In [37]:
# menghilangkan duplicate value pada tabel
df_events = df_events.drop_duplicates(subset=['ID']).reset_index(drop=True)
df_events[df_events["ID"].duplicated(keep=False)]

,index,ID,sku,image_url,url_area,product_ID,brand_ID,category_ID,subcategory_ID,item_ID,current_price,perc.discount,likes_count,is_new,codCountry


# **Data Wrangling**

In [38]:
# menggabungkan tabel products dengan tabel item dengan product_ID sebagai key
df_product_item = df_products.merge(df_item_details, how='left', on='product_ID')
df_product_item.head()

,index_x,product_ID,product_name,index_y,item_ID,variation_0_color,variation_1_color
0,0,1,Chapeau à Visière Anti-UV pour Protection Solaire,0.0,1,Bleu ciel,Gris
1,1,2,Écharpe longue 180cm à pois en broderie pour f...,1.0,2,armée verte,Bleu
2,2,3,Casquette de baseball de sport en plein air de...,2.0,3,#1,#2
3,3,4,Casquettes de Baseball Respirantes Unisexe Par...,3.0,4,rouge,Bleu
4,4,5,LYZA Femmes Châle Surdimensionné Réversible Po...,4.0,5,Rose,Bleu


In [39]:
# menghapus kolom yang dirasa tidak penting
df_product_item.drop(['index_x', 'index_y'], axis=1, inplace=True)
df_product_item.head()

,product_ID,product_name,item_ID,variation_0_color,variation_1_color
0,1,Chapeau à Visière Anti-UV pour Protection Solaire,1,Bleu ciel,Gris
1,2,Écharpe longue 180cm à pois en broderie pour f...,2,armée verte,Bleu
2,3,Casquette de baseball de sport en plein air de...,3,#1,#2
3,4,Casquettes de Baseball Respirantes Unisexe Par...,4,rouge,Bleu
4,5,LYZA Femmes Châle Surdimensionné Réversible Po...,5,Rose,Bleu


In [40]:
df_product_item.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13765 entries, 0 to 13764
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   product_ID         13765 non-null  object
 1   product_name       13765 non-null  object
 2   item_ID            9385 non-null   object
 3   variation_0_color  9385 non-null   object
 4   variation_1_color  9385 non-null   object
dtypes: object(5)
memory usage: 645.2+ KB


In [41]:
# menggabungkan tabel products_item dengan tabel events dengan product_ID sebagai key
df_product_new = df_events.merge(df_product_item, how='left', on='product_ID')
df_product_new.head()

,index,ID,sku,image_url,url_area,product_ID,brand_ID,category_ID,subcategory_ID,item_ID_x,current_price,perc.discount,likes_count,is_new,codCountry,product_name,item_ID_y,variation_0_color,variation_1_color
0,0,27788,SKU585926,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/loskii-wall-art-4619/p-...,4636,120,4,21,4677,21.48 USD,42.0,285,0,"ID,MY,PH,SG,TH,VN",Mesure verticale horizontale de laser de nivea...,NaN,NaN,NaN
1,1,42560,SKU028546,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/plus-size-blouses-and-s...,9862,396,9,98,10372,22.99 USD,65.0,147,0,"ID,MY,PH,SG,TH,VN",Chemises élégantes de dentelle de patchwork,10372,Noir,blanc
2,2,45211,SKU031964,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/beon-vehicle-supplies-5...,3620,246,4,232,3658,1.59 USD,58.0,1,0,"ID,MY,PH,SG,TH,VN",Filtre à essence liquide universel pour essenc...,NaN,NaN,NaN
3,3,48169,SKU037769,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/y-f-m-face-care-tools-5...,1144,73,3,10,1168,19.03 USD,43.0,500,0,"ID,MY,PH,SG,TH,VN",Rouleau de peau YFM pour les rides cicatrices ...,NaN,NaN,NaN
4,4,51841,SKU039460,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/t-shirts-3666/p-51841.html,9315,396,9,427,9761,21.99 USD,0.0,251,0,"ID,MY,PH,SG,TH,VN",Sweat à capuche à manches longues à manches lo...,NaN,NaN,NaN


In [42]:
# menghapus kolom yang dirasa tidak penting
df_product_new.drop(['item_ID_x','item_ID_y'], axis=1, inplace=True)
df_product_new.head()

,index,ID,sku,image_url,url_area,product_ID,brand_ID,category_ID,subcategory_ID,current_price,perc.discount,likes_count,is_new,codCountry,product_name,variation_0_color,variation_1_color
0,0,27788,SKU585926,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/loskii-wall-art-4619/p-...,4636,120,4,21,21.48 USD,42.0,285,0,"ID,MY,PH,SG,TH,VN",Mesure verticale horizontale de laser de nivea...,NaN,NaN
1,1,42560,SKU028546,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/plus-size-blouses-and-s...,9862,396,9,98,22.99 USD,65.0,147,0,"ID,MY,PH,SG,TH,VN",Chemises élégantes de dentelle de patchwork,Noir,blanc
2,2,45211,SKU031964,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/beon-vehicle-supplies-5...,3620,246,4,232,1.59 USD,58.0,1,0,"ID,MY,PH,SG,TH,VN",Filtre à essence liquide universel pour essenc...,NaN,NaN
3,3,48169,SKU037769,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/y-f-m-face-care-tools-5...,1144,73,3,10,19.03 USD,43.0,500,0,"ID,MY,PH,SG,TH,VN",Rouleau de peau YFM pour les rides cicatrices ...,NaN,NaN
4,4,51841,SKU039460,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/t-shirts-3666/p-51841.html,9315,396,9,427,21.99 USD,0.0,251,0,"ID,MY,PH,SG,TH,VN",Sweat à capuche à manches longues à manches lo...,NaN,NaN


In [43]:
df_product_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14792 entries, 0 to 14791
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              14792 non-null  int64  
 1   ID                 14792 non-null  object 
 2   sku                14792 non-null  object 
 3   image_url          14792 non-null  object 
 4   url_area           14792 non-null  object 
 5   product_ID         14792 non-null  object 
 6   brand_ID           14792 non-null  object 
 7   category_ID        14792 non-null  object 
 8   subcategory_ID     14792 non-null  object 
 9   current_price      14792 non-null  object 
 10  perc.discount      14792 non-null  float64
 11  likes_count        14792 non-null  int64  
 12  is_new             14792 non-null  object 
 13  codCountry         14792 non-null  object 
 14  product_name       14792 non-null  object 
 15  variation_0_color  10883 non-null  object 
 16  variation_1_color  108

In [44]:
# menggabungkan tabel brand dengan tabel events dengan brand_ID sebagai key
df_brand_new = df_events.merge(df_brand_details, how='left', on='brand_ID')
df_brand_new.head()

,index_x,ID,sku,image_url,url_area,product_ID,brand_ID,category_ID,subcategory_ID,item_ID,current_price,perc.discount,likes_count,is_new,codCountry,index_y,brand_name
0,0,27788,SKU585926,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/loskii-wall-art-4619/p-...,4636,120,4,21,4677,21.48 USD,42.0,285,0,"ID,MY,PH,SG,TH,VN",119,Loskii
1,1,42560,SKU028546,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/plus-size-blouses-and-s...,9862,396,9,98,10372,22.99 USD,65.0,147,0,"ID,MY,PH,SG,TH,VN",395,ZANZEA
2,2,45211,SKU031964,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/beon-vehicle-supplies-5...,3620,246,4,232,3658,1.59 USD,58.0,1,0,"ID,MY,PH,SG,TH,VN",245,BEON
3,3,48169,SKU037769,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/y-f-m-face-care-tools-5...,1144,73,3,10,1168,19.03 USD,43.0,500,0,"ID,MY,PH,SG,TH,VN",72,Y.F.M
4,4,51841,SKU039460,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/t-shirts-3666/p-51841.html,9315,396,9,427,9761,21.99 USD,0.0,251,0,"ID,MY,PH,SG,TH,VN",395,ZANZEA


In [45]:
# menghapus kolom yang dirasa tidak penting
df_brand_new.drop(['index_x', 'index_y'], axis=1, inplace=True)
df_brand_new.head()

,ID,sku,image_url,url_area,product_ID,brand_ID,category_ID,subcategory_ID,item_ID,current_price,perc.discount,likes_count,is_new,codCountry,brand_name
0,27788,SKU585926,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/loskii-wall-art-4619/p-...,4636,120,4,21,4677,21.48 USD,42.0,285,0,"ID,MY,PH,SG,TH,VN",Loskii
1,42560,SKU028546,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/plus-size-blouses-and-s...,9862,396,9,98,10372,22.99 USD,65.0,147,0,"ID,MY,PH,SG,TH,VN",ZANZEA
2,45211,SKU031964,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/beon-vehicle-supplies-5...,3620,246,4,232,3658,1.59 USD,58.0,1,0,"ID,MY,PH,SG,TH,VN",BEON
3,48169,SKU037769,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/y-f-m-face-care-tools-5...,1144,73,3,10,1168,19.03 USD,43.0,500,0,"ID,MY,PH,SG,TH,VN",Y.F.M
4,51841,SKU039460,https://imgaz1.chiccdn.com/thumb/view/oaupload...,https://fr.newchic.com/t-shirts-3666/p-51841.html,9315,396,9,427,9761,21.99 USD,0.0,251,0,"ID,MY,PH,SG,TH,VN",ZANZEA


In [46]:
df_brand_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12944 entries, 0 to 12943
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              12944 non-null  object 
 1   sku             12944 non-null  object 
 2   image_url       12944 non-null  object 
 3   url_area        12944 non-null  object 
 4   product_ID      12944 non-null  object 
 5   brand_ID        12944 non-null  object 
 6   category_ID     12944 non-null  object 
 7   subcategory_ID  12944 non-null  object 
 8   item_ID         12944 non-null  object 
 9   current_price   12944 non-null  object 
 10  perc.discount   12944 non-null  float64
 11  likes_count     12944 non-null  int64  
 12  is_new          12944 non-null  object 
 13  codCountry      12944 non-null  object 
 14  brand_name      12944 non-null  object 
dtypes: float64(1), int64(1), object(13)
memory usage: 1.6+ MB


In [47]:
# menggabungkan tabel brand_new dengan tabel product_new dengan ID sebagai key
df = df_product_new.merge(df_brand_new, how='inner', on='ID')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14792 entries, 0 to 14791
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              14792 non-null  int64  
 1   ID                 14792 non-null  object 
 2   sku_x              14792 non-null  object 
 3   image_url_x        14792 non-null  object 
 4   url_area_x         14792 non-null  object 
 5   product_ID_x       14792 non-null  object 
 6   brand_ID_x         14792 non-null  object 
 7   category_ID_x      14792 non-null  object 
 8   subcategory_ID_x   14792 non-null  object 
 9   current_price_x    14792 non-null  object 
 10  perc.discount_x    14792 non-null  float64
 11  likes_count_x      14792 non-null  int64  
 12  is_new_x           14792 non-null  object 
 13  codCountry_x       14792 non-null  object 
 14  product_name       14792 non-null  object 
 15  variation_0_color  10883 non-null  object 
 16  variation_1_color  108

In [48]:
# menghapus kolom yang dirasa tidak penting
df.drop(['image_url_x','url_area_x', 'sku_y', 'image_url_y', 'url_area_y', 'product_ID_y', 'brand_ID_y', 'category_ID_y', 'subcategory_ID_y', 'current_price_y', 'perc.discount_y', 'likes_count_y', 'is_new_y', 'codCountry_y'], axis=1, inplace=True)
df.head()

,index,ID,sku_x,product_ID_x,brand_ID_x,category_ID_x,subcategory_ID_x,current_price_x,perc.discount_x,likes_count_x,is_new_x,codCountry_x,product_name,variation_0_color,variation_1_color,item_ID,brand_name
0,0,27788,SKU585926,4636,120,4,21,21.48 USD,42.0,285,0,"ID,MY,PH,SG,TH,VN",Mesure verticale horizontale de laser de nivea...,NaN,NaN,4677,Loskii
1,1,42560,SKU028546,9862,396,9,98,22.99 USD,65.0,147,0,"ID,MY,PH,SG,TH,VN",Chemises élégantes de dentelle de patchwork,Noir,blanc,10372,ZANZEA
2,2,45211,SKU031964,3620,246,4,232,1.59 USD,58.0,1,0,"ID,MY,PH,SG,TH,VN",Filtre à essence liquide universel pour essenc...,NaN,NaN,3658,BEON
3,3,48169,SKU037769,1144,73,3,10,19.03 USD,43.0,500,0,"ID,MY,PH,SG,TH,VN",Rouleau de peau YFM pour les rides cicatrices ...,NaN,NaN,1168,Y.F.M
4,4,51841,SKU039460,9315,396,9,427,21.99 USD,0.0,251,0,"ID,MY,PH,SG,TH,VN",Sweat à capuche à manches longues à manches lo...,NaN,NaN,9761,ZANZEA


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14792 entries, 0 to 14791
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              14792 non-null  int64  
 1   ID                 14792 non-null  object 
 2   sku_x              14792 non-null  object 
 3   product_ID_x       14792 non-null  object 
 4   brand_ID_x         14792 non-null  object 
 5   category_ID_x      14792 non-null  object 
 6   subcategory_ID_x   14792 non-null  object 
 7   current_price_x    14792 non-null  object 
 8   perc.discount_x    14792 non-null  float64
 9   likes_count_x      14792 non-null  int64  
 10  is_new_x           14792 non-null  object 
 11  codCountry_x       14792 non-null  object 
 12  product_name       14792 non-null  object 
 13  variation_0_color  10883 non-null  object 
 14  variation_1_color  10883 non-null  object 
 15  item_ID            14792 non-null  object 
 16  brand_name         147

In [50]:
df_product_new.groupby(by=['is_new', 'product_name'], dropna=False).count()['ID']

is_new  product_name                                                         
0         Bouteille d'huile en acier inoxydable                                  1
          UACY 180cm * 90cm Impression foulard de plage été châle                1
         100pcs / pack de graines d'alto                                         1
         100pcs / sac de graines d'olive                                         1
         12pcs autocollants muraux de miroir sexangle bricolage argent mignon    1
                                                                                ..
1       Vêtement d'intérieur rayé à col en V                                     1
        Vêtement d'intérieur à col en V à imprimé floral                         1
        Vêtements de nuit sans manches respirants unis                           1
        Yoga Pantalon Bloomers Gymnastique                                       1
        Écouteurs intra-auriculaires sans fil Bluetooth                          1
Name: ID,

In [51]:
# mencoba melihat 5 produk teratas
df.sort_values(['product_name'], ascending=False, axis=0, inplace=True)
product_top5 = df.head()
product_top5.head()

,index,ID,sku_x,product_ID_x,brand_ID_x,category_ID_x,subcategory_ID_x,current_price_x,perc.discount_x,likes_count_x,is_new_x,codCountry_x,product_name,variation_0_color,variation_1_color,item_ID,brand_name
6716,7254,1440316,SKUB49125,7990,378,8,353,33.5 USD,49.0,178,0,"ID,MY,PH,SG,TH,VN",Évider étoile sandales romaines,Or,Noir,8278,Lostisy
6835,7377,1447215,SKUB60889,8086,378,8,142,28.4 USD,58.0,221,0,"ID,MY,PH,SG,TH,VN",Évider réglable Soft appartements,gris,marron,8380,Lostisy
8110,8530,1525896,SKUC82984,359,18,2,335,19.06 USD,51.0,493,0,"ID,MY,PH,SG,TH,VN",Évider petit sac de téléphone irrégulière,Black,Brown,362,Brenice
231,247,1048468,SKU391819,3163,61,4,86,5.76 USD,63.0,24,0,TH,Évent d'air de voiture magnétique renforcé en ...,NaN,NaN,3201,BlitzWolf
4186,4626,1341773,SKUA24950,4857,12,4,51,12.99 USD,57.0,7,0,"ID,MY,PH,SG,TH,VN",Étui à crayons en PP transparent de couleur un...,NaN,NaN,4898,XIAOMI


In [55]:
df['ID'].count()

14792

Export ke csv

In [52]:
#df.to_csv(r'/content/drive/MyDrive/Narasio/dataframe.csv', index=False, header=True)